# Simple handling of atom geometries and XYZ-files in python
Python has a very rich ecosystem for all kinds of atomic simulation. `ASE` is somewhat common and probably has the simplest interface, so we will use it here. 

In [1]:
from pathlib import Path  # Builtin library. Utility for file paths.
import numpy as np
import ase
from ase import io
from ase.visualize import view
import ase.build
from ase.data import pubchem

In [5]:
# First, lets say we have some atom locations
xyz = [[0, 0, 0], [1.42, 0, 0]]

# Typically you need to define the cell they live in
cell = np.diag((2.84, 10, 10))

# Now to use the ase i/o functions, an ase Atoms object must be used.
atoms = ase.Atoms("C2", xyz, cell=cell)

# Now you can write these atoms to an xyz file 
# (several formats are supported, but use xyz for easy compatibility with the mathematica nb)
filepath = "my_test_file.xyz"
io.write(filepath, atoms)

# Now lets see the contents of the file we just wrote:
print(Path(filepath).read_text())

2
Lattice="2.84 0.0 0.0 0.0 10.0 0.0 0.0 0.0 10.0" Properties=species:S:1:pos:R:3 pbc="F F F"
C        0.00000000       0.00000000       0.00000000
C        1.42000000       0.00000000       0.00000000



In [3]:
# If you already have a file, you can read it with ase as well:
atoms = io.read(filepath)
# Then you can get the positions:
xyz = atoms.positions
cell = atoms.cell.array
print(xyz)
print(cell)

[[0.   0.   0.  ]
 [1.42 0.   0.  ]]
[[ 2.84  0.    0.  ]
 [ 0.   10.    0.  ]
 [ 0.    0.   10.  ]]


In [4]:
# To clean up any mess, this deletes the previous file again:
p = Path(filepath)
if p.is_file():
    p.unlink()

In [6]:
# You can view atoms with the ase gui:
view(atoms)  # Opens a new window on your computer

# view(atoms, viewer="ngl")  # Requires extra packages. Should show atoms inside the notebook.

In [8]:
# Build simple molecules:
view(ase.build.molecule("C6H6"))

In [7]:
# Look up something online:
view(pubchem.pubchem_atoms_search("anthracene"))

In [9]:
# You can also manually set up a graphene sheet
a = 1.42
n0 = 10
n1 = 12

xyz = np.array([[0, 0, 0], [a, 0, 0]])
s3 = 3**0.5
cell = a / 2 * np.array([[3, -3**0.5, 0], [3, 3**0.5, 0], [0, 0, 10]])

def tile(xyz, cell, ntile, axis):
    xyz = xyz[None, ...].copy()
    xyz = xyz + cell[None, (axis,), :] * np.arange(ntile)[:, None, None]
    cell = cell.copy()
    cell[axis, :] *= ntile
    return xyz.reshape(-1, 3), cell

txyz, tcell = tile(xyz, cell, n0, 0)
txyz, tcell = tile(txyz, tcell, n1, 1)

view(ase.Atoms(["C"]*len(txyz), txyz, cell=tcell))
